# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
!pip install nltk

In [14]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
# load data from database
engine = create_engine('sqlite:///etlpipeline.db')
df = pd.read_sql_table('ETLPipeLine', engine)

In [16]:
df["related"] = df["related"].apply(lambda x: 1 if x == 2 else x)

In [17]:
df.drop(columns = ["child_alone"], inplace = True)

In [18]:
X = df["message"]
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [19]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [20]:
def tokenize(text):
    
    #detect url
    find_url = re.findall(url_regex, text)
    
    #replace url with placeholder
    for url in find_url:
        text = text.replace(url, "urlplaceholder")
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
pipeline_1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
#Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [23]:
# Train Pipeline 1
pipeline_1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
y_pred_1 = pipeline_1.predict(X_test)

In [26]:
print(classification_report(y_test.values, y_pred_1, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5033
               request       0.83      0.41      0.55      1080
                 offer       0.00      0.00      0.00        30
           aid_related       0.73      0.52      0.61      2703
          medical_help       0.58      0.07      0.12       502
      medical_products       0.79      0.07      0.13       305
     search_and_rescue       0.50      0.02      0.04       190
              security       0.50      0.01      0.01       141
              military       0.58      0.06      0.12       231
                 water       0.76      0.24      0.37       415
                  food       0.83      0.30      0.44       740
               shelter       0.81      0.23      0.36       564
              clothing       0.43      0.06      0.11        93
                 money       0.60      0.04      0.07       153
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [28]:
pipeline_2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [29]:
# Train Pipeline 2
pipeline_2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [31]:
#Test Pipeline 2
y_pred_2 = pipeline_2.predict(X_test)

In [32]:
print(classification_report(y_test.values, y_pred_2, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5033
               request       0.74      0.57      0.65      1080
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.59      0.66      2703
          medical_help       0.59      0.25      0.35       502
      medical_products       0.58      0.31      0.40       305
     search_and_rescue       0.83      0.24      0.37       190
              security       0.41      0.06      0.11       141
              military       0.60      0.31      0.41       231
                 water       0.74      0.63      0.68       415
                  food       0.80      0.67      0.73       740
               shelter       0.79      0.52      0.63       564
              clothing       0.62      0.46      0.53        93
                 money       0.60      0.29      0.39       153
        missing_people       0.58      

### 7. Improve your model
Use grid search to find better parameters. 

In [33]:
pipeline_2.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_p

In [34]:
parameters = {'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000)}

cv = GridSearchCV(pipeline_2, param_grid=parameters, cv = 2, scoring='f1_micro', n_jobs=-1, verbose = 3)

In [35]:
# Training Grid Search Model on Pipeline 2
cv.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), score=0.6556270858789416, total= 2.2min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 1), score=0.6612853600709463, total= 2.2min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.4min remaining:    0.0s


[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2), score=0.6549850681826358, total= 4.6min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=None, features__text_pipeline__vect__ngram_range=(1, 2), score=0.6590864606131781, total= 3.9min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 1), score=0.6561963548705213, total= 2.0min
[CV] features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vec

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 50.9min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.75, 1.0), 'features__text_pipeline__vect__max_features': (None, 5000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=3)

In [36]:
cv.best_params_

{'features__text_pipeline__vect__max_df': 0.75,
 'features__text_pipeline__vect__max_features': None,
 'features__text_pipeline__vect__ngram_range': (1, 1)}

### 8. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [38]:
y_pred_3 = cv.predict(X_test)

In [39]:
print(classification_report(y_test.values, y_pred_3, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5033
               request       0.75      0.57      0.65      1080
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.60      0.66      2703
          medical_help       0.59      0.25      0.35       502
      medical_products       0.58      0.31      0.40       305
     search_and_rescue       0.75      0.24      0.36       190
              security       0.30      0.04      0.07       141
              military       0.57      0.32      0.41       231
                 water       0.72      0.65      0.68       415
                  food       0.79      0.68      0.73       740
               shelter       0.78      0.52      0.62       564
              clothing       0.66      0.43      0.52        93
                 money       0.62      0.27      0.38       153
        missing_people       0.57      

### 9. Export your model as a pickle file

In [40]:
pickle.dump(cv, open('clf_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.